<a href="https://colab.research.google.com/github/mohithxoxo/python/blob/master/Foursquare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [2]:
CLIENT_ID = 'GJE0I1OSMRPDIGFYRJBT5V30MPRPJZB4PKPPDRW5KPKQQZAF' # your Foursquare ID
CLIENT_SECRET = '1AQKBNKEOE1WRBIO2S5I2ZPZCZ3MNYQTJYXEIZ34PJVEUDNA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GJE0I1OSMRPDIGFYRJBT5V30MPRPJZB4PKPPDRW5KPKQQZAF
CLIENT_SECRET:1AQKBNKEOE1WRBIO2S5I2ZPZCZ3MNYQTJYXEIZ34PJVEUDNA


#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [44]:
address = 'Bangalore,Karnataka,India'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

12.9791198 77.5912997


In [45]:
search_query = 'restaurants'
radius = 100000
print(search_query + ' .... OK!')

restaurants .... OK!


In [46]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GJE0I1OSMRPDIGFYRJBT5V30MPRPJZB4PKPPDRW5KPKQQZAF&client_secret=1AQKBNKEOE1WRBIO2S5I2ZPZCZ3MNYQTJYXEIZ34PJVEUDNA&ll=12.9791198,77.5912997&v=20180604&query=restaurants&radius=100000&limit=30'

#### Send the GET Request and examine the results

In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e746c8d963d29001bc6aef2'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'primary': True,
      'shortName': 'Indian'}],
    'hasPerk': False,
    'id': '4dc55eafd4c0ad9c0f8585e5',
    'location': {'address': 'Above Mom & me',
     'cc': 'IN',
     'city': 'Bangalore',
     'country': 'India',
     'crossStreet': '1st cross, koramangala 5th block',
     'distance': 6109,
     'formattedAddress': ['Above Mom & me (1st cross, koramangala 5th block)',
      'Bangalore',
      'Karnātaka',
      'India'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 12.933695361984093,
       'lng': 77.6229044540145}],
     'lat': 12.933695361984093,
     'lng': 77.6229044540145,
     'state': 'Karnātaka'},
    'name': 'Turquoise Restaurant

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [48]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode
0,4dc55eafd4c0ad9c0f8585e5,Turquoise Restaurants,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1584688542,False,Above Mom & me,"1st cross, koramangala 5th block",12.933695,77.622904,"[{'label': 'display', 'lat': 12.93369536198409...",6109,IN,Bangalore,Karnātaka,India,"[Above Mom & me (1st cross, koramangala 5th bl...",NaN
1,4f1b8bdde4b0817d3ce12ae2,Sagar Restaurants And Caterers,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1584688542,False,NaN,NaN,12.973076,77.540214,"[{'label': 'display', 'lat': 12.97307583179569...",5582,IN,Bangalore,Karnātaka,India,"[Bangalore, Karnātaka, India]",NaN
2,5acd8445c6666607b18c6c80,Rajesh Restaurants,"[{'id': '54135bf5e4b08f3d2429dfe0', 'name': 'M...",v-1584688542,False,"L 141, 5th Main Rd, Opposite Rajesh Jewellery ...",NaN,12.914153,77.632150,"[{'label': 'display', 'lat': 12.914153, 'lng':...",8481,IN,Bangalore,Karnātaka,India,"[L 141, 5th Main Rd, Opposite Rajesh Jewellery...",560102
3,5a1418b964c8e11573ea2bd3,Nandhana Restaurants,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1584688542,False,"No.7,4th B block,Near BDA Complex, Koramangala",Near BDA Complex,12.936221,77.621676,"[{'label': 'display', 'lat': 12.93622058555660...",5802,IN,Bangalore,Karnātaka,India,"[No.7,4th B block,Near BDA Complex, Koramangal...",560034
4,5201fb13498e98c02b3ed4bf,sinbad Restaurants,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1584688542,False,Pyramid Food Court,NaN,12.950709,77.642395,"[{'label': 'display', 'lat': 12.95070934295654...",6381,IN,Bangalore,Karnātaka,India,"[Pyramid Food Court, Bangalore, Karnātaka, India]",NaN


#### Define information of interest and filter dataframe

In [49]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,id
0,Turquoise Restaurants,Indian Restaurant,Above Mom & me,"1st cross, koramangala 5th block",12.933695,77.622904,"[{'label': 'display', 'lat': 12.93369536198409...",6109,IN,Bangalore,Karnātaka,India,"[Above Mom & me (1st cross, koramangala 5th bl...",NaN,4dc55eafd4c0ad9c0f8585e5
1,Sagar Restaurants And Caterers,Fast Food Restaurant,NaN,NaN,12.973076,77.540214,"[{'label': 'display', 'lat': 12.97307583179569...",5582,IN,Bangalore,Karnātaka,India,"[Bangalore, Karnātaka, India]",NaN,4f1b8bdde4b0817d3ce12ae2
2,Rajesh Restaurants,Multicuisine Indian Restaurant,"L 141, 5th Main Rd, Opposite Rajesh Jewellery ...",NaN,12.914153,77.632150,"[{'label': 'display', 'lat': 12.914153, 'lng':...",8481,IN,Bangalore,Karnātaka,India,"[L 141, 5th Main Rd, Opposite Rajesh Jewellery...",560102,5acd8445c6666607b18c6c80
3,Nandhana Restaurants,Indian Restaurant,"No.7,4th B block,Near BDA Complex, Koramangala",Near BDA Complex,12.936221,77.621676,"[{'label': 'display', 'lat': 12.93622058555660...",5802,IN,Bangalore,Karnātaka,India,"[No.7,4th B block,Near BDA Complex, Koramangal...",560034,5a1418b964c8e11573ea2bd3
4,sinbad Restaurants,Indian Restaurant,Pyramid Food Court,NaN,12.950709,77.642395,"[{'label': 'display', 'lat': 12.95070934295654...",6381,IN,Bangalore,Karnātaka,India,"[Pyramid Food Court, Bangalore, Karnātaka, India]",NaN,5201fb13498e98c02b3ed4bf
5,Inchara,Indian Restaurant,Vijaya Bank Colony Extension,Outer Ring Road,13.017228,77.660399,"[{'label': 'display', 'lat': 13.01722807351631...",8612,IN,Bangalore,Karnātaka,India,[Vijaya Bank Colony Extension (Outer Ring Road...,NaN,5162dc55e4b00e8fc41461df
6,Public,Lounge,NaN,NaN,12.899199,77.605490,"[{'label': 'display', 'lat': 12.899199, 'lng':...",9028,IN,Bangalore,Karnātaka,India,"[Bangalore 560076, Karnātaka, India]",560076,593c07e9724750423ee53e03
7,Shanmukha Restaurants,Hyderabadi Restaurant,NaN,NaN,12.983856,77.749790,"[{'label': 'display', 'lat': 12.983856, 'lng':...",17200,IN,Bangalore,Karnātaka,India,"[Bangalore 560066, Karnātaka, India]",560066,5843cc5b5a58693fa1e1916d
8,Karnataka Vanashree Restaurants,Indian Restaurant,NaN,NaN,12.801905,77.577029,"[{'label': 'display', 'lat': 12.80190537859610...",19787,IN,NaN,NaN,India,[India],NaN,52bd16a111d24cfb8962d505




```
# This is formatted as code
```

#### Let's visualize the ATM that are nearby


In [50]:
dataframe_filtered.name

0              Turquoise Restaurants
1     Sagar Restaurants And Caterers
2                 Rajesh Restaurants
3               Nandhana Restaurants
4                 sinbad Restaurants
5                            Inchara
6                             Public
7              Shanmukha Restaurants
8    Karnataka Vanashree Restaurants
Name: name, dtype: object

In [52]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around 

# add a red circle marker to represent 
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Restaurants',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the ATMS as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>

## 2. Explore a Given Venue
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_

In [54]:
venue_id = '4dc55eafd4c0ad9c0f8585e5' # ID of Turquoise Restaurants
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4dc55eafd4c0ad9c0f8585e5?client_id=GJE0I1OSMRPDIGFYRJBT5V30MPRPJZB4PKPPDRW5KPKQQZAF&client_secret=1AQKBNKEOE1WRBIO2S5I2ZPZCZ3MNYQTJYXEIZ34PJVEUDNA&v=20180604'

#### Send GET request for result

In [55]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'storeId', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '₹₹', 'priceTier': 2}],
    'name': 'Price',
    'summary': '₹₹',
    'type': 'price'}]},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'bestPhoto': {'createdAt': 1360851096,
  'height': 540,
  'id': '511cf098e4b0ca541c754131',
  'prefix': 'https://fastly.4sqi.net/img/general/',
  'source': {'name': 'Foursquare for Android',
   'url': 'https://foursquare.com/download/#/android'},
  'suffix': '/31574185_xK2I7rXSTfEyVmvFZJLRyN99vzCVMkVA20B2L3nQn2c.jpg',
  'visibility': 'public',
  'width': 960},
 'canonicalUrl': 'https://foursquare.com/v/turquoise-restaurants/4dc55eafd4c0ad9c0f8585e5',
 'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
    'suffix': '.png'},
   'id': '4bf58dd8d48988d10f941735',
   'name': 'Indian Restaurant',
   'pluralName': 'Indian Restaurants',
   'primar

### B. Get the venue's overall rating

In [56]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.8


Since this restaurant has a slightly better rating, let's explore it further.

### C. Get the number of tips

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

#### Create URL and send GET request. Make sure to set limit to get all tips

In [57]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
venue_id = '4dc55eafd4c0ad9c0f8585e5' #used ny restaurant id for better results
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e746df90f5968001b36c3b2'},
 'response': {'tips': {'count': 12,
   'items': [{'agreeCount': 0,
     'canonicalUrl': 'https://foursquare.com/item/50d7d593e4b0e253420fa2e1',
     'createdAt': 1356322195,
     'disagreeCount': 0,
     'id': '50d7d593e4b0e253420fa2e1',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'text': 'Love the food esp lamb biryani. Also enjoyed nalli gosht, peshawari paneer tikka( creamy n melt in mouth) dal bukhara- awesome, haleem, peshawari chicken perfectly spiced. Baklawa superb! Must visit.',
     'todo': {'count': 0},
     'type': 'user',
     'user': {'firstName': 'Navaz',
      'id': '16307970',
      'lastName': 'L',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/SYG4GDPTG1TWUMXA.jpg'}}},
    {'agreeCount': 0,
     'canonicalUrl': 'https://foursquare.com/item/4e46ac0b814d84e76a9a5fb3',
     'createdAt': 1313254411,
     'disagreeCount': 0,
     

#### Get tips and list of associated features

In [60]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

#### Format column width and display all tips

In [61]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Love the food esp lamb biryani. Also enjoyed nalli gosht, peshawari paneer tikka( creamy n melt in mouth) dal bukhara- awesome, haleem, peshawari chicken perfectly spiced. Baklawa superb! Must visit.",0,0,50d7d593e4b0e253420fa2e1,Navaz,L,NaN,16307970
1,Indian/Parsi. Decent Galawati Kabab and excellent Baida Roti for starters. Zardari Kofta and Murgh Makhani with the awesome Naan Baluchistan or Khameeri Roti. Many dishes unavailable. MFT Rs.1200,0,0,4e46ac0b814d84e76a9a5fb3,manu,p,NaN,320944


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.

<a id="item3"></a>

## 3. Search a Foursquare User
> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### Define URL, send GET request and display features associated with user

In [72]:
user_id = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

KeyError: ignored

In [68]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

KeyError: ignored

#### How many tips has this user submitted?

In [73]:
user_data['tips']

KeyError: ignored

Wow! So it turns out that Nick is a very active Foursquare user, with more than 250 tips.

### Get User's tips

In [0]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

#### Let's get the venue for the tip with the greatest number of agree counts

In [0]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

### Get User's friends

In [0]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

Interesting. Despite being very active, it turns out that Nick does not have any friends on Foursquare. This might definitely change in the future.

### Retrieve the User's Profile Image

In [0]:
user_data

In [0]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://igx.4sqi.net/img/user/300x300/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg')

<a id="item4"></a>

## 4. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

#### So, if you are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of karavalli Restaurant bangalore.

In [0]:
latitude = 12.9724
longitude = 77.6086

#### Define URL

In [76]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=GJE0I1OSMRPDIGFYRJBT5V30MPRPJZB4PKPPDRW5KPKQQZAF&client_secret=1AQKBNKEOE1WRBIO2S5I2ZPZCZ3MNYQTJYXEIZ34PJVEUDNA&ll=12.9724,77.6086&v=20180604&radius=100000&limit=30'

#### Send GET request and examine results

In [0]:
import requests

In [79]:
results = requests.get(url).json()
'There are {} around karavalli restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around karavalli restaurant.'

#### Get relevant part of JSON

In [80]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4b530849f964a520108d27e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'primary': True,
    'shortName': 'Hotel'}],
  'id': '4b530849f964a520108d27e3',
  'location': {'address': '37-39',
   'cc': 'IN',
   'city': 'Bangalore',
   'country': 'India',
   'crossStreet': 'Mahatma Gandhi Road',
   'distance': 1057,
   'formattedAddress': ['37-39 (Mahatma Gandhi Road)',
    'Bangalore 560001',
    'Karnātaka',
    'India'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 12.973457334267048,
     'lng': 77.61828850236205}],
   'lat': 12.973457334267048,
   'lng': 77.61828850236205,
   'postalCode': '560001',
   'state': 'Karnātaka'},
  'name': 'The Obero

#### Process JSON and convert it to a clean dataframe

In [81]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,The Oberoi,Hotel,37-39,Mahatma Gandhi Road,12.973457,77.618289,"[{'label': 'display', 'lat': 12.973457334267048, 'lng': 77.61828850236205}]",1057,560001,IN,Bangalore,Karnātaka,India,"[37-39 (Mahatma Gandhi Road), Bangalore 560001, Karnātaka, India]",NaN,4b530849f964a520108d27e3
1,Harima,Sushi Restaurant,"4th Floor, Devatha Plaza",Residency Road,12.967755,77.600073,"[{'label': 'display', 'lat': 12.967754615278329, 'lng': 77.60007264264614}]",1059,NaN,IN,Bangalore,Karnātaka,India,"[4th Floor, Devatha Plaza (Residency Road), Bangalore, Karnātaka, India]",NaN,4b97ad0bf964a5200c0f35e3
2,Arbor Brewing Company,Brewery,"8, 3rd Flr., Allied Grande Plaza, Magrath Rd.",MG Rd.,12.970140,77.610857,"[{'label': 'display', 'lat': 12.970139599197337, 'lng': 77.61085658133437}]",351,560025,IN,Bangalore,Karnātaka,India,"[8, 3rd Flr., Allied Grande Plaza, Magrath Rd. (MG Rd.), Bangalore 560025, Karnātaka, India]",NaN,50b8cfebe4b0b45a1a4919ad
3,Cubbon Park,Park,M G Road,NaN,12.977042,77.595277,"[{'label': 'display', 'lat': 12.977042175203783, 'lng': 77.59527683258057}]",1534,NaN,IN,Bangalore,Karnātaka,India,"[M G Road, Bangalore, Karnātaka, India]",NaN,4bbc4513e45295217db855a4
4,JW Marriott Hotel Bengaluru,Hotel,24/1 Vittal Mallya Road,NaN,12.972362,77.595051,"[{'label': 'display', 'lat': 12.972361772490226, 'lng': 77.59505132638924}]",1469,560001,IN,Bangalore,Karnātaka,India,"[24/1 Vittal Mallya Road, Bangalore 560001, Karnātaka, India]",NaN,51d1245e498ef93fd0e713bb
5,Hard Rock Cafe Bengaluru,American Restaurant,40 St. Marks Road,St. Marks Rd,12.976389,77.601468,"[{'label': 'display', 'lat': 12.9763894, 'lng': 77.6014683}]",892,560001,IN,Bangalore,Karnātaka,India,"[40 St. Marks Road (St. Marks Rd), Bangalore 560001, Karnātaka, India]",NaN,4b5aaef9f964a520b8d028e3
6,UB City,Shopping Mall,at Vittal Mallya Rd,NaN,12.971709,77.595905,"[{'label': 'display', 'lat': 12.971708980695311, 'lng': 77.59590528412356}]",1379,560001,IN,Bangalore,Karnātaka,India,"[at Vittal Mallya Rd, Bangalore 560001, Karnātaka, India]",NaN,4bcd805cfb84c9b61512223e
7,Truffles - Ice & Spice,Burger Joint,#22,St. Marks Rd,12.971802,77.601031,"[{'label': 'display', 'lat': 12.971801622357315, 'lng': 77.60103059863826}]",823,560001,IN,Bangalore,Karnātaka,India,"[#22 (St. Marks Rd), Bangalore 560001, Karnātaka, India]",NaN,4bd446b341b9ef3b27a801e6
8,Toscano,Italian Restaurant,UB City Level 2 Concorde Block,24 Vittal Mallya Road,12.971980,77.596066,"[{'label': 'display', 'lat': 12.97198038085137, 'lng': 77.59606565238312}]",1360,NaN,IN,Bangalore,Karnātaka,India,"[UB City Level 2 Concorde Block (24 Vittal Mallya Road), Bangalore, Karnātaka, India]",NaN,4bc1cd90b492d13a4e74a660
9,Taj West End,Hotel,Racecourse Rd.,NaN,12.984572,77.584893,"[{'label': 'display', 'lat': 12.984572066608893, 'lng': 77.58489294114838}]",2906,NaN,IN,Bangalore,Karnātaka,India,"[Racecourse Rd., Bangalore, Karnātaka, India]",NaN,4b69572ff964a520509f2be3


#### Let's visualize these items on the map around our location

In [83]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='karavalli',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<a id="item5"></a>

## 5. Explore Trending Venues
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

#### Now, instead of simply exploring the area around karavalli, you are interested in knowing the venues that are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. So let's do that and get the trending venues around karavalli.

In [85]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e74710695feaf001b0c84e7'},
 'response': {'venues': []}}

### Check if any venues are trending at this time

In [0]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [87]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 

### Visualize trending venues

In [0]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [89]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

<a id="item6"></a>